In [ ]:
import pandas as pd # Pandas para tratar los datos
import dash # Framework para crear aplicaciones web interactivas
from dash import dcc, html # dcc para crear desplegables
from dash.dependencies import Input, Output
import plotly.express as px # Para graficos de barras y línea
import plotly.graph_objs as go
import os # Para acceder a archivos en carpetas


In [21]:
# Cargar los datos
data_folder = "C:/Users/comod/Downloads/Visualizador de datos ENSU 2024/data"
# Inicializar una lista vacía para almacenar los DataFrames
dfs = []

# Diccionario para mapear los códigos de trimestre a meses
trimestre_a_mes = {
    '0324': 'Marzo',
    '0624': 'Junio',
    '0924': 'Septiembre',
    '1224': 'Diciembre'
}

# Recorrer los archivos en la carpeta
for file in os.listdir(data_folder):
    if file.startswith("conjunto_de_datos_ensu_") and file.endswith(".csv"):
        # Cargar cada archivo CSV en un DataFrame
        df = pd.read_csv(os.path.join(data_folder, file))
        # Extraer el trimestre del nombre del archivo
        trimestre = file.split("_")[-1].split(".")[0]
        df['Trimestre'] = trimestre
        # Agregar la columna del mes
        df['Mes'] = trimestre_a_mes.get(trimestre, 'Desconocido')
        dfs.append(df)

# Combinar todos los DataFrames en uno solo
df_combined = pd.concat(dfs, ignore_index=True)

C:\Users\comod\AppData\Local\Temp\ipykernel_13708\1198153767.py:18: DtypeWarning:

Columns (144) have mixed types. Specify dtype option on import or set low_memory=False.

C:\Users\comod\AppData\Local\Temp\ipykernel_13708\1198153767.py:18: DtypeWarning:

Columns (148) have mixed types. Specify dtype option on import or set low_memory=False.

C:\Users\comod\AppData\Local\Temp\ipykernel_13708\1198153767.py:18: DtypeWarning:

Columns (244) have mixed types. Specify dtype option on import or set low_memory=False.



In [ ]:
# Filtrar datos para Nuevo León
df_nl = df_combined[df_combined['NOM_ENT'] == 'NUEVO LEON']
df_nl.columns.values #Data test para verificar que se están todas las columnas completas

array(['ID_VIV', 'ID_PER', 'UPM', 'VIV_SEL', 'R_SEL', 'CVE_ENT',
       'NOM_ENT', 'CVE_MUN', 'NOM_MUN', 'LOC', 'CD', 'NOM_CD', 'PER',
       'R_DEF', 'H_MUD', 'S_MUD', 'TIPO_CUEST', 'N_ENT', 'SEXO', 'EDAD',
       'BP1_1', 'BP1_2_01', 'BP1_2_02', 'BP1_2_03', 'BP1_2_04',
       'BP1_2_05', 'BP1_2_06', 'BP1_2_07', 'BP1_2_08', 'BP1_2_09',
       'BP1_2_10', 'BP1_2_11', 'BP1_2_12', 'BP1_3', 'BP1_4_1', 'BP1_4_2',
       'BP1_4_3', 'BP1_4_4', 'BP1_4_5', 'BP1_4_6', 'BP1_4_7', 'BP1_4_8',
       'BP1_5_1', 'BP1_5_2', 'BP1_5_3', 'BP1_5_4', 'BP1_5_5', 'BP1_6_01',
       'BP1_6_02', 'BP1_6_03', 'BP1_6_04', 'BP1_6_05', 'BP1_6_06',
       'BP1_6_07', 'BP1_6_08', 'BP1_6_09', 'BP1_6_10', 'BP1_6_11',
       'BP1_6_12', 'BP1_6_13', 'BP1_6_99', 'BP1_7_1', 'BP1_7_2',
       'BP1_7_3', 'BP1_7_4', 'BP1_7_5', 'BP1_7_6', 'BP1_8_1', 'BP1_9_1',
       'BP1_8_2', 'BP1_9_2', 'BP1_8_3', 'BP1_9_3', 'BP1_8_4', 'BP1_9_4',
       'BP1_8_5', 'BP1_9_5', 'BP1_8_6', 'BP1_9_6', 'BP2_1', 'BP2_2_01',
       'BP2_2_02', 'BP2

In [ ]:
# Aplicación Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Visualizador ENSU - Nuevo León 2024"), # Encabezado del visualizador
    dcc.Dropdown(
        id='visualization-selector',
        options=[
            {'label': 'Percepción de inseguridad por mes', 'value': 'time'},
            {'label': 'Municipios con mayor y menor percepción', 'value': 'municipalities'},
            {'label': 'Percepción por tipo de lugar', 'value': 'place'},
            {'label': 'Perspectiva de género', 'value': 'gender'}
        ],
        value='time'
    ),
    dcc.Graph(id='main-graph')
])


#Aquí filtro el indicador BP1_1 a solo 2 ya que son las respuestas de inseguridad y calculo el porcentaje de quienes respondieron eso
@app.callback(
    Output('main-graph', 'figure'),
    Input('visualization-selector', 'value')
)
def update_graph(selected_visualization):
    if selected_visualization == 'time':
        df_time = df_nl.groupby('Mes').apply(lambda x: (x['BP1_1'] == 2).sum() / len(x) * 100).reset_index() # Aquí filtro el indicador BP1_1 a solo 2 ya que son las respuestas de inseguridad y calculo el porcentaje de quienes respondieron eso
        df_time.columns = ['Mes', 'Porcentaje_Inseguridad']
        df_time['Mes'] = pd.Categorical(df_time['Mes'], categories=['Marzo', 'Junio', 'Septiembre', 'Diciembre'], ordered=True) # Ordeno el mes para que no se muestre alfabeticamente en el gráfico
        df_time = df_time.sort_values('Mes') #En caso de agregar mas años a analizar agregaria mes y años ordenados en la lista Ex. Dic24, Mar25, jun25, etc
        fig = px.line(df_time, x='Mes', y='Porcentaje_Inseguridad', title='Percepción de inseguridad por mes en 2024') 
        fig.update_layout(xaxis_title="Mes", yaxis_title="Porcentaje de percepción de inseguridad", yaxis=dict(range=[0, 100])) # Agregué ejes de 0 a 100 para evitar sesgos
    elif selected_visualization == 'municipalities':
        df_mun = df_nl.groupby('NOM_MUN').apply(lambda x: (x['BP1_1'] == 2).sum() / len(x) * 100).sort_values(ascending=False).reset_index()
        df_mun.columns = ['Municipio', 'Porcentaje_Inseguridad']
        fig = px.bar(df_mun, x='Municipio', y='Porcentaje_Inseguridad', title='Percepción de inseguridad por municipio')
        fig.update_layout(xaxis_title="Municipio", yaxis_title="Porcentaje de percepción de inseguridad")
    elif selected_visualization == 'place':
        place_columns = ['BP1_4_1', 'BP1_4_2', 'BP1_4_3', 'BP1_4_4', 'BP1_4_5', 'BP1_4_6', 'BP1_4_7', 'BP1_4_8']
        place_names = ['Casa', 'Trabajo', 'Calle', 'Escuela', 'Mercado', 'Centro comercial', 'Banco', 'Transporte público']
        df_place = df_nl[place_columns].apply(lambda x: (x == 2).sum() / len(x) * 100).reset_index()
        df_place.columns = ['Lugar', 'Porcentaje_Inseguridad']
        df_place['Lugar'] = place_names
        fig = px.bar(df_place, x='Lugar', y='Porcentaje_Inseguridad', title='Percepción de inseguridad por tipo de lugar')
        fig.update_layout(xaxis_title="Lugar", yaxis_title="Porcentaje de percepción de inseguridad")
    else:  # gender
        df_gender = df_nl.groupby('SEXO').apply(lambda x: (x['BP1_1'] == 2).sum() / len(x) * 100).reset_index()
        df_gender.columns = ['SEXO', 'Porcentaje_Inseguridad']
        df_gender['SEXO'] = df_gender['SEXO'].map({1: 'Hombre', 2: 'Mujer'})
        fig = px.bar(df_gender, x='SEXO', y='Porcentaje_Inseguridad', title='Percepción de inseguridad por género')
        fig.update_layout(xaxis_title="Género", yaxis_title="Porcentaje de percepción de inseguridad")
    return fig


if __name__ == '__main__':
    app.run_server(debug=True)


C:\Users\comod\AppData\Local\Temp\ipykernel_13708\1774569342.py:27: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

